In [3]:
from dotenv import load_dotenv
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.openai import OpenAI

from let_history_talk.models import PodcastResponse

load_dotenv()

True

In [84]:
# tokenizer = Anthropic(model="claude-3-7-sonnet-latest").tokenizer
# Settings.tokenizer = tokenizer

# llm = Anthropic(model="claude-3-7-sonnet-latest")

# Settings.llm = llm

llm = OpenAI(
    model='gpt-4o'
)

In [71]:
# System prompt for the podcasting engine 

# Copyright Open-Notebooklm

SYSTEM_PROMPT_PODCAST_GENERATION = """
You are a world-class podcast producer tasked with transforming the provided context into an engaging and informative podcast script. Your goal is to extract the most interesting and insightful content for a compelling podcast discussion.

# Steps to Follow:

1. **Analyze the Input:**
   Carefully examine the provided context, identifying key topics, points, and interesting facts or anecdotes that could drive an engaging podcast conversation. Disregard irrelevant information or formatting issues.

2. **Brainstorm Ideas:**
   In the scratchpad, creatively brainstorm ways to present the key points engagingly. Consider:
   - Analogies, storytelling techniques, or hypothetical scenarios to make content relatable
   - Ways to make complex topics accessible to a general audience
   - Thought-provoking questions to explore during the podcast
   - Creative approaches to fill any gaps in the information

3. **Craft the Dialogue:**
   Develop a natural, conversational flow between the host (Alicia) and the guest speaker (the author or an expert on the topic). Incorporate:
   - The best ideas from your brainstorming session
   - Clear explanations of complex topics
   - An engaging and lively tone to captivate listeners
   - A balance of information and entertainment

   Rules for the dialogue:
   - The host (Alicia) always initiates the conversation and interviews the guest
   - Include thoughtful questions from the host to guide the discussion
   - Incorporate natural speech patterns, including occasional verbal fillers (e.g., "um," "well," "you know")
   - Allow for natural interruptions and back-and-forth between host and guest
   - Ensure the guest's responses are substantiated by the input text, avoiding unsupported claims
   - Avoid any marketing or self-promotional content from the guest
   - The host concludes the conversation

4. **Summarize Key Insights:**
   Naturally weave a summary of key points into the closing part of the dialogue. This should feel like a casual conversation rather than a formal recap, reinforcing the main takeaways before signing off.

5. **Maintain Authenticity:**
   Throughout the script, strive for authenticity in the conversation. Include:
   - Moments of genuine curiosity or surprise from the host
   - Instances where the guest might briefly struggle to articulate a complex idea
   - Light-hearted moments or humor when appropriate
   - Brief personal anecdotes or examples that relate to the topic (within the bounds of the input text)

6. **Consider Pacing and Structure:**
   Ensure the dialogue has a natural flow:
   - Start with a strong hook to grab the listener's attention
   - Gradually build complexity as the conversation progresses
   - Include brief "breather" moments for listeners to absorb complex information
   - End on a high note, perhaps with a thought-provoking question or a call-to-action for listeners

IMPORTANT: Always reply given the following format: {structured_output}
"""

PODCAST_PROMPT_TEMPLATE = """
   Generate a podcat script about {topic}. 
   
   This is the context: {context}.

   You must return a valid JSON object with both 'scratchpad' and 'podcast_script' fields.
   The 'podcast_script' field is required and must have 'title', 'host', 'guest', and 'dialogue' properties.
   
   Example format:
   {structured_output}
"""

In [72]:
with open('out/summary.txt', 'r') as f:
    summary = f.read()

In [86]:
message_templates = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=SYSTEM_PROMPT_PODCAST_GENERATION,
    ),
    ChatMessage(
        content=PODCAST_PROMPT_TEMPLATE,
        role=MessageRole.USER,
    )
]

chat_template = ChatPromptTemplate(message_templates=message_templates)

In [87]:
# sllm = llm.as_structured_llm(output_cls=PodcastResponse)

# response = sllm.chat(messages=messages)

response = llm.structured_predict(
    output_cls=PodcastResponse,
    prompt=chat_template,
    structured_output=PodcastResponse.model_json_schema(),
    topic="World War II", 
    context=summary
)

In [89]:
response.podcast_script

PodcastScript(title='World War II: A Global Conflict That Shaped Modern History', host='Alicia', guest='Dr. John Smith', dialogue=[{'role': 'host', 'message': "Hello, everyone, and welcome to today's episode. I'm Alicia, and joining me is Dr. John Smith, a historian specializing in World War II. Dr. Smith, thank you for being here."}, {'role': 'guest', 'message': "Thank you, Alicia. It's a pleasure to be here and discuss such a pivotal moment in history."}, {'role': 'host', 'message': 'World War II is often described as the deadliest conflict in human history. Can you give us a sense of its scale and impact?'}, {'role': 'guest', 'message': 'Absolutely. World War II was truly global, involving over 30 countries and resulting in an estimated 70-85 million deaths. It reshaped borders, economies, and societies worldwide.'}, {'role': 'host', 'message': "The war began with Germany's invasion of Poland in 1939. Can you tell us about the early stages and how the Blitzkrieg tactics played a rol

In [ ]:
import pickle

pickle.dump(response, open('out/podcast.pkl', 'wb'))